## Deep Dream:

Here list the basic algorithm stream of the deep dream:

1. Input a image and we read it as a numpy array of float numbers
2. Pass the numpy array into trained models (Inception model from google)
3. Update the image numpy array based on the gradient of certain layer from model
4. Repeat 3. until we like it to stop (choosing a stop time.)

Inception模型是在相当低分辨率的图像上进行训练的，大概200-300像素。所以，当我们使用更大分辨率的图像时，DeepDream算法会在图像中创建许多小的图案。 
一个解决方案是将输入图像缩小到200-300像素。但是这么低的分辨率（的结果）是像素化而且丑陋的。

那么另外一种方案就是逐层缩小原始图，然后再小图上运行deep dream，然后再up scale输出回去：

Input - > Downscale  ->  Small Image 1-> ... Downscale -> Small Graph n
             |                 |                    |               |
             V                 V                    V               V
Output<-----Upscale  <----- DeepDream   <--------Upscale   <------Deepdream                        



In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random
import math

# Image manipulation.
import PIL.Image
from scipy.ndimage.filters import gaussian_filter

# Download the model:
import inception5h
import download

inception5h.maybe_download()

Data has apparently already been downloaded and unpacked.


以上我们把将会需要用到的包调用完了，最后*inception5h.py*是一个下载inception model的脚本，载入模型的话只需要:

In [9]:
model = inception5h.Inception5h()
len(model.layer_tensors) # Inception5h.py Inception5h() listed 12 usually used layers 
model.layer_tensors

[<tf.Tensor 'conv2d0:0' shape=(?, ?, ?, 64) dtype=float32>,
 <tf.Tensor 'conv2d1:0' shape=(?, ?, ?, 64) dtype=float32>,
 <tf.Tensor 'conv2d2:0' shape=(?, ?, ?, 192) dtype=float32>,
 <tf.Tensor 'mixed3a:0' shape=(?, ?, ?, 256) dtype=float32>,
 <tf.Tensor 'mixed3b:0' shape=(?, ?, ?, 480) dtype=float32>,
 <tf.Tensor 'mixed4a:0' shape=(?, ?, ?, 508) dtype=float32>,
 <tf.Tensor 'mixed4b:0' shape=(?, ?, ?, 512) dtype=float32>,
 <tf.Tensor 'mixed4c:0' shape=(?, ?, ?, 512) dtype=float32>,
 <tf.Tensor 'mixed4d:0' shape=(?, ?, ?, 528) dtype=float32>,
 <tf.Tensor 'mixed4e:0' shape=(?, ?, ?, 832) dtype=float32>,
 <tf.Tensor 'mixed5a:0' shape=(?, ?, ?, 832) dtype=float32>,
 <tf.Tensor 'mixed5b:0' shape=(?, ?, ?, 1024) dtype=float32>]

As we can see that all the 12 layers are 